In [9]:
import warnings
warnings.filterwarnings('ignore')
import networkx as nx
import numpy as np

In [3]:
from Bio.PDB import PDBList
import os

# Download PDB file from RCSB
def download_pdb(pdb_id):
    pdb_list = PDBList()
    pdb_list.retrieve_pdb_file(pdb_id, pdir='./PDBs', file_format="pdb")
    os.rename(f'./PDBs/pdb{pdb_id}.ent', f'./PDBs/{pdb_id}.pdb')
download_pdb('1fc2')

In [10]:
from graphein.protein.config import ProteinGraphConfig
from graphein.protein.edges.atomic import add_atomic_edges
params_to_change = {"granularity": "atom", "edge_construction_functions": [add_atomic_edges]}

config = ProteinGraphConfig(**params_to_change)
config.dict()

{'granularity': 'atom',
 'keep_hets': [],
 'insertions': True,
 'alt_locs': 'max_occupancy',
 'pdb_dir': None,
 'verbose': False,
 'exclude_waters': True,
 'deprotonate': False,
 'protein_df_processing_functions': None,
 'edge_construction_functions': [<function graphein.protein.edges.atomic.add_atomic_edges(G: networkx.classes.graph.Graph, tolerance: float = 0.56) -> networkx.classes.graph.Graph>],
 'node_metadata_functions': [<function graphein.protein.features.nodes.amino_acid.meiler_embedding(n: str, d: Dict[str, Any], return_array: bool = False) -> Union[pandas.core.series.Series, numpy.ndarray]>],
 'edge_metadata_functions': None,
 'graph_metadata_functions': None,
 'get_contacts_config': None,
 'dssp_config': None}

In [2]:
from graphein.protein.graphs import construct_graph

graph = construct_graph(config=config, pdb_code="1FC2")

NameError: name 'config' is not defined

In [ ]:
# поиск координат СА мутирующего остатка, пример D178A C


res = convert_to_three_letter('D')
mut = '178'
chain = 'C'

for node, data in graph.nodes(data=True):
    if f'{chain}:{res}:{mut}:CA' in node:
        mut_center = data['coords']

In [ ]:
from graphein.protein.visualisation import plotly_protein_structure_graph
from graphein.protein.subgraphs import extract_subgraph_from_point


# выделение подграфа вокруг мутируемого остатка
subgraph = extract_subgraph_from_point(graph, centre_point=(mut_center), radius=10)

plotly_protein_structure_graph(subgraph, node_size_min=4, node_size_multiplier=2)

NameError: name 'graph' is not defined

In [1]:
from graphein.protein.utils import save_graph_to_pdb
from graphein.protein.graphs import read_pdb_to_dataframe

pdb_id = '1ahw'
save_graph_to_pdb(subgraph, f'./cutPDBs/{pdb_id}_cut.pdb')

pdb_path=f'./cutPDBs/1ahw_cut.pdb'
df = read_pdb_to_dataframe(pdb_path)
df

NameError: name 'subgraph' is not defined

In [ ]:
# из датафрейма вытащить матрицу расстояний
from graphein.molecule.edges.distance import compute_distmat
coords = df.filter(like='_coord')
coords

distMatrix = compute_distmat(coords)
distMatrix

array([[0.00000000e+00, 1.27269360e+00, 1.25755557e+00, ...,
        1.60493830e+01, 1.83720041e+01, 1.69634346e+01],
       [1.27269360e+00, 0.00000000e+00, 2.22531818e+00, ...,
        1.60176225e+01, 1.80948968e+01, 1.67239032e+01],
       [1.25755557e+00, 2.22531818e+00, 0.00000000e+00, ...,
        1.56269151e+01, 1.82722909e+01, 1.69115330e+01],
       ...,
       [1.60493830e+01, 1.60176225e+01, 1.56269151e+01, ...,
        9.53674316e-07, 5.14692588e+00, 5.05378957e+00],
       [1.83720041e+01, 1.80948968e+01, 1.82722909e+01, ...,
        5.14692588e+00, 0.00000000e+00, 1.86091832e+00],
       [1.69634346e+01, 1.67239032e+01, 1.69115330e+01, ...,
        5.05378957e+00, 1.86091832e+00, 0.00000000e+00]])

In [ ]:
# потом эту матрицу подаем на вход функции которая выдает массив атомов которые находятся друг от друга на расстоянии r
from graphein.molecule.edges.distance import get_interacting_atoms

np.set_printoptions(threshold=200)

def get_interacting_atoms(angstroms: float, distmat: pd.DataFrame):
    """Find the atoms that are within a particular radius of one another."""
    return np.where(distmat <= angstroms, 0, 1)

residue_environment_dist = 10
Dmin = 0
Dstep = 2
Dmax = residue_environment_dist



arr = get_interacting_atoms(10, distMatrix)

from IPython.display import display
import numpy as np
frequency = np.sum(arr[arr != 0]) / 2
print(frequency)

# Пример создания и вывода большого массива
display(arr)


NameError: name 'np' is not defined

In [ ]:
from graphein.protein.features.nodes.amino_acid import load_expasy_scales()
graphein.protein.features.nodes.amino_acid.expasy_protein_scale

(array([0, 1]), array([0, 0]))

In [ ]:
import graphein
graphein.molecule.features.graph.molecule.mol_descriptors()